In [55]:
import cv2
import numpy as np
import os

def split_video_into_frames(video_path, output_folder, num_frames=20):
    # 비디오를 로드하고 총 프레임 수를 구함
    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(total_frames)
    
    # 균일한 간격으로 프레임을 선택하기 위한 간격 계산
    interval = total_frames // num_frames
    
    # 출력 폴더 생성 (존재하지 않을 경우)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 선택된 프레임을 이미지 파일로 저장
    for i in range(num_frames):
        # 비디오를 올바른 위치로 이동
        video.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        
        # 프레임 읽기
        success, frame = video.read()
        if not success:
            break  # 읽기 실패시 중단
        
        # 이미지 파일로 저장
        cv2.imwrite(f"{output_folder}/frame_{i+1:03d}.jpg", frame)
    
    # 자원 해제
    video.release()
    print(f"Saved {num_frames} frames to {output_folder}")

for i in range(17, 18):
    video_path = f'dataset/fortest/{i}.mp4'  # 비디오 파일 경로
    output_folder = f'output_frames/unlabeled/{i}'  # 출력될 폴더
    split_video_into_frames(video_path, output_folder)

232
Saved 20 frames to output_frames/unlabeled/17


In [39]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
import json
# MoveNet 모델 로드
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4").signatures['serving_default']

def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_with_pad(image, 192, 192)
    image = tf.expand_dims(image, axis=0)
    image = tf.cast(image, dtype=tf.int32)
    return image

def get_keypoints(image_path):
    image = load_and_preprocess_image(image_path)
    keypoints = movenet(image)['output_0']
    keypoints = tf.squeeze(keypoints, axis=0)
    return keypoints.numpy().flatten()

In [54]:
base_dir = 'output_frames\\unlabeled\\17'  # 기본 디렉토리 설정

sequences = []

sequence_keypoints = []

for i in range(1, 21):  # 1부터 20까지 반복
    frame_filename = f'frame_{i:03d}.jpg'  # 001, 002, ... 형태로 포매팅
    image_path = os.path.join(base_dir, frame_filename)
    keypoints = get_keypoints(image_path)
    sequence_keypoints.append(keypoints)
    
sequences.append(sequence_keypoints)

sequences = np.array(sequences)

sequences_list = sequences.tolist()

# JSON 파일로 저장
data = {'sequences': sequences_list}
with open('unlabeled_data.json', 'w') as json_file:
    json.dump(data, json_file)